In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister, execute, Aer, IBMQ,BasicAer
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
import math
import matplotlib.pyplot as plt
from random import *

In [2]:
#calcul le nombre de 1 dans la représentation binaire du nombre de qubit
def countOnes(numberOfQubits):
    counts = 0
    while numberOfQubits!=0:
        numberOfQubits=numberOfQubits&(numberOfQubits-1)
        counts=counts+1
    return counts

In [52]:
#permet de construire les entreés du circuit 
#et calcul le nombre de bit auxiliaire recquis

def buildInputs(numberOfQubits,valueA,valueB):
    
    #useful constant
    logn=math.floor(math.log(numberOfQubits,2))
    log2_3n=math.floor(math.log((2/3)*numberOfQubits,2))
    stateVectorSize=int(math.pow(2,numberOfQubits))
    
    n_auxillaryQubits=numberOfQubits-countOnes(numberOfQubits)-logn
    
    if(n_auxillaryQubits<=0):
        n_auxillaryQubits=1
    
    
    #Definition of the registers

    # These registers contains the numbers to add
    A = QuantumRegister(numberOfQubits,"a")
    circ_A=QuantumCircuit(A)
    B = QuantumRegister(numberOfQubits,"b")
    circ_B=QuantumCircuit(B)
    #Sum register
    S=QuantumRegister(numberOfQubits+1,"s")
    #Ancillary register
    Anc=QuantumRegister(n_auxillaryQubits,"anc")
    circ_S=QuantumCircuit(S,Anc)
    
    #initializing circuit
    state_A = [0]*stateVectorSize #initialize to 0
    state_B = [0]*stateVectorSize #initialize to 0
    state_A[valueA-1]=1
    state_B[valueB-1]=1
    circ_A.initialize(state_A,A)
    circ_B.initialize(state_B,B)
    
    Adder = circ_A+circ_B+circ_S
    
    
    #Utility list
    Buffer=[]  #on y place les P successifs
    P_list=[]
    P_inv_list=[]

    
    #compute g[i,i+1] into auxillary register for 0<i<n
    for i in range(0,numberOfQubits):
        Adder.ccx(A[i],B[i],S[i+1])
        
    Adder.barrier()    
    #compute p[i,i+1] into b register for 0<i<n
    for i in range(1,numberOfQubits):
        Adder.cx(A[i],B[i])
    
    Adder.barrier()
    

    
    #P-round
    i=0
    for t in range(1,logn):
        tmp=math.pow(2,t)
        roundUpperBound = math.floor(numberOfQubits/(tmp))
        for m in range(1,roundUpperBound):
            if t>1:
                Adder.ccx(P_list[t-2][2*m-1],P_list[t-2][2*m],Anc[i])
                Buffer.append(Anc[i])
                P_inv_list.append([P_list[t-2][2*m-1],P_list[t-2][2*m],Anc[i]])
                
            else:
                Adder.ccx(B[2*m],B[2*m+1],Anc[i])
                Buffer.append(Anc[i])
                P_inv_list.append([B[2*m],B[2*m+1],Anc[i]])
            i=i+1
        P_list.append(Buffer)
        Buffer=[]
    Adder.barrier()
    
    
    
    #G-round
    for t in range(1,logn+1):
        tmp=int(math.pow(2,t))
        tmp1=int(math.pow(2,t-1))
        roundUpperBound = math.floor(numberOfQubits/(tmp))
        for m in range(0,roundUpperBound):
            if t>1:
                Adder.ccx(S[tmp*m+tmp1],P_list[t-2][2*m],S[tmp*m+tmp])
            else:
                Adder.ccx(S[tmp*m+tmp1],B[2*m+1],S[tmp*m+tmp])
    Adder.barrier()
    
              
    #C-round
    for t in range(log2_3n,0,-1):
        tmp=int(math.pow(2,t))
        tmp1=int(math.pow(2,t-1))
        roundUpperBound=math.floor((numberOfQubits-tmp1)/tmp)
        for m in range (1,roundUpperBound+1):
            if t>1:
                Adder.ccx(S[tmp*m],P_list[t-2][2*m-1],S[tmp*m+tmp1])
            else:
                Adder.ccx(S[tmp*m],B[2*m],S[tmp*m+tmp1])
    
    Adder.barrier()
    
    
    for m in range(len(P_inv_list)-1,-1,-1):
        Adder.ccx(P_inv_list[m][0],P_inv_list[m][1],P_inv_list[m][2])
                
    Adder.barrier()
    
    for i in range(0,numberOfQubits):
        Adder.cx(B[i],S[i])
    
    Adder.barrier()
    
    Adder.cx(A[0],S[0])
    
    for i in range(1,numberOfQubits):
        Adder.cx(A[i],B[i])
        
        
    #adding a  Register to measure the state of the qubits
    outS = ClassicalRegister(numberOfQubits+1,"outS")

    #adding measure circuit
    meas = QuantumCircuit(S,outS)
    meas.barrier(S)
    meas.measure(S,outS)  #measure S values into outS
    qc = Adder + meas   #concatenate the circuit
    
    return qc

In [27]:
def fromBinaryStringToInt(binString):
    n=binString[0]
    bitString=[]
    bitInt=0
    a=0
    for i in range(0,len(n)):
        if(n[i]!=" "):
            bitString.append(n[i])
    for i in range(len(bitString)-1,-1,-1):
        if(bitString[i]=="1"):
            bitInt=bitInt+int(math.pow(2,a))
        a=a+1
    return(bitInt)

In [28]:

#Test bench
def testBench():
    nbTest=0
    backendQASM = BasicAer.get_backend('qasm_simulator')
    for i in range(1,8):
        bound = math.pow(2,i)
        stateVectorSize=int(bound)
        for j in range(0,int(stateVectorSize/2)):
            valueA = randint(1,stateVectorSize)
            valueB = randint(1,stateVectorSize)
            Adder = buildInputs(i,valueA,valueB)
            job = execute(Adder,backendQASM,shots=1)
            result = job.result()
            counts = result.get_counts(Adder)
            nbTest=nbTest+1
            print(nbTest)
            if(valueA+valueB-2!=fromBinaryStringToInt(list(counts.keys()))):
                print("ERROR WRONG SUM !")
                print(valueA)
                print(valueB)
                print(i)
                return
    print("TEST COMPLETE")

In [54]:
#testBench()
#A=54
#B=36
Adder=buildInputs(20,A+1,B+1)
Adder.depth()

14

In [40]:
backendQASM = BasicAer.get_backend('qasm_simulator')
job = execute(Adder,backendQASM,shots=1)
result = job.result()
counts = result.get_counts(Adder)
print(A+B)
print(counts) 

90
{'1011010': 1}
